# Laboratorio de regresión - 4

|                |   |
:----------------|---|
| **Nombre**     | Jesús Emmanuel Flores Cortés  |
| **Fecha**      | 11/09/2025  |
| **Expediente** | 751571  |

## Modelos penalizados

Hasta ahora la función de costo que usamos para decidir qué tan bueno es nuestro modelo al momento de ajustar es:

$$ \text{RSS} = \sum_{i=1}^n e_i^2 = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$

Dado que los errores obtenidos son una combinación de sesgo y varianza, puede ser que se sesgue un parámetro para minimizar el error. Esto significa que el modelo puede decidir que la salida no sea una combinación de los factores, sino una fuerte predilección sobre uno de los factores solamente. 

E.g. se quiere ajustar un modelo

$$ \hat{z} = \hat{\beta_0} + \hat{\beta_1} x + \hat{\beta_2} y $$

Se ajusta el modelo y se decide que la mejor decisión es $\hat{\beta_1} = 10000$ y $\hat{\beta_2}=50$. Considera limitaciones de problemas reales:
- Quizás los parámetros son ajustes de maquinaria que se deben realizar para conseguir el mejor producto posible, y que $10000$ sea imposible de asignar.
- Quizás los datos actuales están sesgados y sólo hacen parecer que uno de los factores importa más que el otro.

Una de las formas en las que se puede mitigar este problema es penalizando a los parámetros del modelo, cambiando la función de costo:

$$ \text{RSS}_{L2} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p \hat{\beta_j}^2 $$

El *L2* significa que se está agregando una penalización de segundo orden. Lo que hace esta penalización es que los factores ahora sólo tendrán permitido crecer si hay una reducción al menos proporcional en el error (sacrificamos sesgo, pero reducimos la varianza).

Asimismo, existe la penalización *L1*

$$ \text{RSS}_{L1} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p |\hat{\beta_j}| $$

A las penalizaciones *L2* y *L1* se les conoce también como Ridge y Lasso, respectivamente.

Para realizar una regresión con penalización de Ridge o de Lasso usamos el objeto `Ridge(alpha=?)` o `Lasso(alpha=?)` en lugar de `LinearRegression()` de `sklearn`.

Utiliza el dataset de publicidad (Advertising.csv) y realiza 3 regresiones múltiples:

$$ \text{sales} = \beta_0 + \beta_1 (\text{TV}) + \beta_2 (\text{radio}) + \beta_3 (\text{newspaper}) + \epsilon $$

1. Sin penalización
2. Con penalización L2
3. Con penalización L1

Compara los resultados de los parámetros y sus *p-values*, y los $R^2$ resultantes.

In [4]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score
import statsmodels.api as sm

In [7]:
df = pd.read_csv('Advertising.csv')

In [8]:
df.head()

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


In [9]:
X = df[["TV", "radio", "newspaper"]]
y = df["sales"]

## SIN PENALIZACION

In [17]:
X_const = sm.add_constant(X)  
ols_model = sm.OLS(y, X_const).fit()

In [18]:
print("Resultados OLS (sin penalización):")
print(ols_model.summary())

Resultados OLS (sin penalización):
                            OLS Regression Results                            
Dep. Variable:                  sales   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     570.3
Date:                Thu, 11 Sep 2025   Prob (F-statistic):           1.58e-96
Time:                        14:26:01   Log-Likelihood:                -386.18
No. Observations:                 200   AIC:                             780.4
Df Residuals:                     196   BIC:                             793.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.

El modelo de regresión lineal (OLS) tiene los siguientes resultados:
R² = 0.897, lo que indica que el modelo explica el 89.7% de la variabilidad en las ventas.
Los coeficientes fueron:
  - TV: 0.0458 
  - radio: 0.1885
  - newspaper: -0.0010

El intercepto fue de 2.9389
  
Esto significa que TV y radio son variables con fuerte relación positiva sobre las ventas, mientras que newspaper no tiene efecto significativo.


## CON PENALIZACION L2

In [12]:
ridge_model = Ridge(alpha=1.0)  
ridge_model.fit(X, y)
y_pred_ridge = ridge_model.predict(X)
r2_ridge = r2_score(y, y_pred_ridge)

In [13]:
print("\nResultados Ridge (L2):")
print("Coeficientes:", ridge_model.coef_)
print("Intercepto:", ridge_model.intercept_)
print("R2:", r2_ridge)


Resultados Ridge (L2):
Coeficientes: [ 0.04576464  0.1885251  -0.00103629]
Intercepto: 2.938967458330149
R2: 0.8972106380074802


El modelo Ridge (penalización L2):
R² = 0.8972, muy similar al de OLS.
Los coeficientes fueron:
  - TV: 0.04576  
  - radio: 0.18852  
  - newspaper: -0.00103  

Intercepto: 2.9389  

La penalización L2 encogió ligeramente los coeficientes, el modelo prácticamente no cambió respecto a OLS.

## CON PENALIZACION L1

In [14]:
lasso_model = Lasso(alpha=0.1)
lasso_model.fit(X, y)
y_pred_lasso = lasso_model.predict(X)
r2_lasso = r2_score(y, y_pred_lasso)

In [16]:
print("\nResultados Lasso (L1):")
print("Coeficientes:", lasso_model.coef_)
print("Intercepto:", lasso_model.intercept_)
print("R^2:", r2_lasso)



Resultados Lasso (L1):
Coeficientes: [ 0.04575172  0.18788735 -0.00066758]
Intercepto: 2.9444386596073855
R^2: 0.8972068586756202


El modelo Lasso (penalización L1):
R² = 0.8972, de nuevo prácticamente igual a OLS y Ridge.
Los coeficientes fueron:
  - TV: 0.04575  
  - radio: 0.18789  
  - newspaper: -0.00067  
- Intercepto: 2.9444

La penalización L1 redujo aún más el coeficiente de newspaper, acercándolo prácticamente a cero, pero mantuvo TV y radio como predictores principales.


## CONCLUSION

Al comparar los tres modelos vemos que:
- OLS, Ridge y Lasso alcanzan un desempeño prácticamente idéntico en términos de R², lo cual indica que el modelo explica bien la variabilidad de las ventas.
- Los p-values en OLS muestran que TV y radio son altamente significativos, mientras que newspaper no
- Esto confirma que los modelos penalizados, aunque no mejoran el ajuste en este caso, ayudan a simplificar mejor las variables realmente importantes.